In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200615.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],Rank your movie choices. You don't have to do a full ranking. [Seventh Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice],What theme should we explore next week? [Eighth Choice],What theme should we explore next week? [Nineth Choice]
0,6/14/2020 15:24:39,Woman in the Dunes,Body Double,Crash,The Housemaid,Obsessed,Swimfan,NaN,No Theme/Free-for-all,Coming of Age,Cuban Cinema,Monsters,Cocaine,COMPUTER: Friend or Foe?,Movies with Slime,Hardboiled Action,<60% on Rotten Tomatoes
1,6/14/2020 15:30:52,Body Heat,Obsessed,Body Double,The Housemaid,Crash,Swimfan,Woman in the Dunes,Cocaine,COMPUTER: Friend or Foe?,Movies with Slime,<60% on Rotten Tomatoes,Coming of Age,Cuban Cinema,Hardboiled Action,Monsters,NaN
2,6/14/2020 15:50:45,Swimfan,Obsessed,Body Heat,Crash,NaN,NaN,NaN,COMPUTER: Friend or Foe?,Movies with Slime,Cocaine,<60% on Rotten Tomatoes,Coming of Age,Cuban Cinema,Hardboiled Action,Monsters,No Theme/Free-for-all
3,6/14/2020 15:52:43,Swimfan,The Housemaid,Body Heat,Woman in the Dunes,Obsessed,Crash,Body Double,<60% on Rotten Tomatoes,COMPUTER: Friend or Foe?,Cocaine,Monsters,NaN,NaN,NaN,NaN,NaN
4,6/14/2020 16:05:32,Woman in the Dunes,Crash,Body Heat,Obsessed,Swimfan,Body Double,The Housemaid,Cocaine,Hardboiled Action,Cuban Cinema,Movies with Slime,Monsters,COMPUTER: Friend or Foe?,Coming of Age,<60% on Rotten Tomatoes,No Theme/Free-for-all
5,6/14/2020 16:21:35,Crash,Swimfan,Body Heat,Woman in the Dunes,Body Double,The Housemaid,Obsessed,Movies with Slime,Cocaine,COMPUTER: Friend or Foe?,<60% on Rotten Tomatoes,Hardboiled Action,Cuban Cinema,Monsters,Coming of Age,No Theme/Free-for-all
6,6/14/2020 18:16:12,Obsessed,NaN,NaN,NaN,NaN,NaN,NaN,No Theme/Free-for-all,Coming of Age,<60% on Rotten Tomatoes,COMPUTER: Friend or Foe?,NaN,NaN,NaN,NaN,NaN
7,6/14/2020 18:51:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Theme/Free-for-all,COMPUTER: Friend or Foe?,Coming of Age,NaN,NaN,NaN,NaN,NaN,NaN
8,6/14/2020 22:06:04,Crash,The Housemaid,Obsessed,Body Heat,Swimfan,Woman in the Dunes,Body Double,COMPUTER: Friend or Foe?,<60% on Rotten Tomatoes,Cocaine,Hardboiled Action,Movies with Slime,Monsters,Coming of Age,Cuban Cinema,No Theme/Free-for-all


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Body Heat', 'Obsessed', 'Body Double', 'The Housemaid', 'Crash',
       'Swimfan', 'Woman in the Dunes'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Body Heat': <Candidate('Body Heat')>,
 'Obsessed': <Candidate('Obsessed')>,
 'Body Double': <Candidate('Body Double')>,
 'The Housemaid': <Candidate('The Housemaid')>,
 'Crash': <Candidate('Crash')>,
 'Swimfan': <Candidate('Swimfan')>,
 'Woman in the Dunes': <Candidate('Woman in the Dunes')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Woman in the Dunes, Body Double, Crash, The Housemaid, Obsessed, Swimfan)>,
 <Ballot(Body Heat, Obsessed, Body Double, The Housemaid, Crash, Swimfan, Woman in the Dunes)>,
 <Ballot(Swimfan, Obsessed, Body Heat, Crash)>,
 <Ballot(Swimfan, The Housemaid, Body Heat, Woman in the Dunes, Obsessed, Crash, Body Double)>,
 <Ballot(Woman in the Dunes, Crash, Body Heat, Obsessed, Swimfan, Body Double, The Housemaid)>,
 <Ballot(Crash, Swimfan, Body Heat, Woman in the Dunes, Body Double, The Housemaid, Obsessed)>,
 <Ballot(Obsessed)>,
 <Ballot()>,
 <Ballot(Crash, The Housemaid, Obsessed, Body Heat, Swimfan, Woman in the Dunes, Body Double)>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate             Votes  Status
------------------  -------  --------
Crash                     2  Hopeful
Swimfan                   2  Hopeful
Woman in the Dunes        2  Hopeful
Obsessed                  1  Rejected
Body Heat                 1  Rejected
The Housemaid             0  Rejected
Body Double               0  Rejected
Blank votes               1  Rejected

ROUND 2
Candidate             Votes  Status
------------------  -------  --------
Crash                     3  Hopeful
Swimfan                   2  Hopeful
Woman in the Dunes        2  Rejected
Obsessed                  0  Rejected
Body Heat                 0  Rejected
The Housemaid             0  Rejected
Body Double               0  Rejected
Blank votes               2  Rejected

FINAL RESULT
Candidate             Votes  Status
------------------  -------  --------
Crash                     5  Elected
Swimfan                   2  Rejected
Woman in the Dunes        0  Rejected
Obsessed                  0  Re

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                   Votes  Status
------------------------  -------  --------
No Theme/Free-for-all           3  Hopeful
COMPUTER: Friend or Foe?        2  Hopeful
Cocaine                         2  Hopeful
<60% on Rotten Tomatoes         1  Rejected
Movies with Slime               1  Rejected
Coming of Age                   0  Rejected
Hardboiled Action               0  Rejected
Cuban Cinema                    0  Rejected
Monsters                        0  Rejected

ROUND 2
Candidate                   Votes  Status
------------------------  -------  --------
COMPUTER: Friend or Foe?        3  Hopeful
Cocaine                         3  Hopeful
No Theme/Free-for-all           3  Rejected
<60% on Rotten Tomatoes         0  Rejected
Movies with Slime               0  Rejected
Coming of Age                   0  Rejected
Hardboiled Action               0  Rejected
Cuban Cinema                    0  Rejected
Monsters                        0  Rejected

FINAL RESULT
Candidate 